In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## Estimación del variograma experimental


In [ ]:
# Importamos módulos para el análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gstools as gs

# Posibles modelos de variograma
models = {
    "Gaussian": gs.Gaussian,
    "Exponential": gs.Exponential,
    "Stable": gs.Stable,
    "Rational": gs.Rational,
    "Circular": gs.Circular,
    "Spherical": gs.Spherical,
    "SuperSpherical": gs.SuperSpherical,
    "JBessel": gs.JBessel,
}

In [ ]:
# ESPEFICICAMOS ARCHIVO EXCEL Y HOJA
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
FILE = "data/sondeos.xlsx"
HOJA = "ej_Au"
XCOL = "Dist. (pies)"
VCOL = "Conc. (dtws)"
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

data = pd.read_excel(FILE, HOJA, engine = "openpyxl")
xi = data[XCOL]
val = data[VCOL]*-1
data.describe()

In [ ]:
# ESTIMACIÓN DEL VARIOGRAMA EXPERIMENTAL
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
bins = None
bins = np.arange(0, 60, 5)
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

coords = [xi]
bin_center, gamma = gs.vario_estimate(coords, val, bins)
fig, ax = plt.subplots()
ax.scatter(bin_center, gamma)
ax.set_ylabel("$\gamma$")
ax.set_xlabel("Distance")
ax.set_ylim(ymin=0, ymax=20)
res = ax.set_xlim(xmin=0)

In [ ]:
# AJUSTE MANUAL DE UN MODELO DE VARIOGRAMA
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
C = 9.5
a = 18
Co= 7.5
MODELO = "Spherical"
XMAX = 60
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

fit_model = models[MODELO](dim=1, var=C, len_scale=a, nugget=Co)
plt.scatter(bin_center, gamma, color="k", label="data")
ax = plt.gca()
res = fit_model.plot(x_max=XMAX, ax=ax)
ax.set_ylabel("$\gamma$")
ax.set_xlabel("Distance")
ax.set_ylim(ymin=0)
res = ax.set_xlim(xmin=0)

In [ ]:
# AJUSTE AUTOMÁTICO DEL MODELO DE VARIOGRAMA
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
PEPITA = True
MODELO = "Spherical"
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

fit_model = models[MODELO](dim=1, nugget=PEPITA)
para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True)
plt.scatter(bin_center, gamma, color="k", label="data")
ax = plt.gca()
fit_model.plot(x_max=XMAX, ax=ax)
ax.set_ylabel("$\gamma$")
ax.set_xlabel("Distance")
ax.set_ylim(ymin=0)
ax.set_xlim(xmin=0)

print("PARÁMETROS DE AJUSTE")
print("="*20)
print("C: {0:.3f}".format(para["var"]))
print("a: {0:.3f}".format(para["len_scale"]))
print("Co: {0:.3f}".format(para["nugget"]))
print("R2: {0:.3f}".format(r2))

In [ ]:
# ESTIMACIÓN DEL MEJOR MODELO DE AJUSTE

# Representamos datos del variograma experimental
plt.scatter(bin_center, gamma, color="k", label="data")
ax = plt.gca()

# Ajustamos todos los modelos
scores = []
parameters = []
for model in models:
    try:
        fit_model = models[model](dim=2)
        para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True)
        var, len_scale, nugget = para["var"], para["len_scale"], para["nugget"]
        parameters.append([var, len_scale, nugget])
        fit_model.plot(x_max=XMAX, ax=ax)
        scores.append(r2)
    except:
        scores.append(0.000)
        parameters.append([0.0, 0.0, 0.0])


parameters = np.array(parameters)
scores = np.array(scores)
pos_scores = np.argsort(scores*-1)
modelos = np.array(list(models.keys()))[pos_scores]
parameters = parameters[pos_scores]
scores = scores[pos_scores]

for n in np.arange(scores.size):
    if scores[n] > 0:
        print("{0} {1} R2: {2:.3f} [C: {3:.2f}, a: {4:.2f}, Co: {5:.2f}]".format(n+1, modelos[n], scores[n], parameters[n,0], 
                                      parameters[n,1], parameters[n,2]))
    else:
        print("{0} {1} -- The model cannot be adjusted  --".format(n+1, modelos[n]))